In [ ]:
import os.path
from skimage.transform import resize
import imageio
import numpy as np
from keras.utils import np_utils
import pandas as pd
from sklearn.utils import shuffle
from keras import optimizers
import cv2
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
def decompose_line(line):
    decompose = line.split('|')
    tokens = []
    for str in decompose:
        if len(str) > 1:
            tokens.append(str.strip())
    return tokens

# Create object to transform data
data_generator = ImageDataGenerator(rotation_range=90,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    shear_range=0.1,
                                    zoom_range=0.1,
                                    horizontal_flip=True,
                                    fill_mode='nearest',
                                    brightness_range=[0.75, 1],
                                    channel_shift_range = 0.25)

def getTransform(Image):
    Image = Image.reshape((1,) + Image.shape)  
    # Apply transformation
    for batch in data_generator.flow(Image):
        return batch[0]

In [ ]:
data_dir = 'D:\OpenVINO\PH2Dataset'
im_size = (50, 75, 3)

diagnosis_file = open(os.path.join(data_dir, 'PH2_dataset.txt'), 'r')
header = decompose_line(diagnosis_file.readline())
t = []
for line in diagnosis_file:
    if "||" in line:
        t.append(decompose_line(line))        
diagnosis_file.close()
df_dataset = pd.DataFrame(t, columns = header)

generator = 10
samples = len(df_dataset['Name'])
X = np.ndarray(shape=(samples*generator, im_size[0], im_size[1], im_size[2]))
y = np.ndarray(samples*generator, )
names = []

i = 0
for index in np.random.permutation(samples):
    im_file = df_dataset['Name'][index]
    file_path = os.path.join(data_dir, 'PH2 Dataset images', im_file, im_file + '_Dermoscopic_Image', im_file + '.bmp')
    image = imageio.imread(file_path)
    image = resize(image, im_size, mode='constant', anti_aliasing=False)
    for j in range(0, generator):
        X[i] = getTransform(image)
        y[i] = df_dataset['Clinical Diagnosis'][index]
        names.append(df_dataset['Name'][index])
        i = i + 1

In [ ]:
samples = X.shape[0]
to_train = int(0.75*samples)
x_train = X[:to_train]
x_test = X[to_train:]

y_train = y[:to_train]
y_test = y[to_train:]
names_test = names[to_train:]

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

num_classes = y_train.shape[1]

print('Number of classes: ' + str(num_classes))
print('Number of training samples: ' + str(to_train))
print('Number of test samples:', x_test.shape[0])

In [ ]:
from matplotlib.pyplot import imshow
test = np.random.randint(x_test.shape[0])
imshow(x_test[test])
plt.title(names_test[test] + ' diagnosis: ' + str(y_test[test][1]))

In [ ]:
from keras import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras import backend as K
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils.vis_utils import plot_model
import matplotlib.image as mpimg

def VGG19():

    model = Sequential()

    model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu', input_shape = im_size, padding = 'same'))
    model.add(Conv2D(64, (3, 3), activation = 'relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size = (2, 2)))

    model.add(Conv2D(128, (3, 3), activation = 'relu', padding = 'same'))
    model.add(Conv2D(128, (3, 3), activation = 'relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    
    model.add(Conv2D(256, (3, 3), activation = 'relu', padding = 'same'))
    model.add(Conv2D(256, (3, 3), activation = 'relu', padding = 'same'))
    model.add(Conv2D(256, (3, 3), activation = 'relu', padding = 'same'))
    #model.add(Conv2D(256, (3, 3), activation = 'relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size = (2, 2)))

    model.add(Conv2D(512, (3, 3), activation = 'relu', padding = 'same'))
    model.add(Conv2D(512, (3, 3), activation = 'relu', padding = 'same'))
    model.add(Conv2D(512, (3, 3), activation = 'relu', padding = 'same'))
    #model.add(Conv2D(512, (3, 3), activation = 'relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size = (2, 2)))

    model.add(Conv2D(512, (3, 3), activation = 'relu', padding = 'same'))
    model.add(Conv2D(512, (3, 3), activation = 'relu', padding = 'same'))
    model.add(Conv2D(512, (3, 3), activation = 'relu', padding = 'same'))
    #model.add(Conv2D(512, (3, 3), activation = 'relu', padding = 'same'))

    model.add(Flatten())
    model.add(Dense(4096, activation = 'relu'))
    model.add(Dense(4096, activation = 'relu'))
    model.add(Dense(1000, activation = 'relu'))
    model.add(Dense(num_classes, activation = 'softmax'))
    
    return model

In [ ]:
adam = optimizers.Adam(lr = 0.09, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, decay = 0.0)

model = VGG19()
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.01), metrics = ['accuracy'])

print(model.summary())
plot_model(model, to_file = 'D:\OpenVINO\cellsVGG.png', show_shapes = True, show_layer_names = True)
#img = mpimg.imread('D:\OpenVINO\cellsVGG.png')
#plt.figure(figsize = (30, 60))
#imgplot = plt.imshow(img)

In [ ]:
batch_size = 20
epochs = 2
history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, verbose = 1, validation_data = (x_test, y_test))

score = model.evaluate(x_test, y_test, verbose = 0)
print('Test loss:', score[0])
print('Test accuracy:', score[1]) 

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)

line1 = plt.plot(epochs, val_loss_values, label = 'Validation/Test Loss')
line2 = plt.plot(epochs, loss_values, label = 'Training Loss')
plt.setp(line1, linewidth = 2.0, marker = '+', markersize = 10.0)
plt.setp(line2, linewidth = 2.0, marker = '4', markersize = 10.0)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
history_dict = history.history

loss_values = history_dict['acc']
val_loss_values = history_dict['val_acc']
epochs = range(1, len(loss_values) + 1)

line1 = plt.plot(epochs, val_loss_values, label = 'Validation/Test Accuracy')
line2 = plt.plot(epochs, loss_values, label = 'Training Loss')
plt.setp(line1, linewidth = 2.0, marker = '+', markersize = 10.0)
plt.setp(line2, linewidth = 2.0, marker = '4', markersize = 10.0)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
model.save('D:\OpenVINO\cellsvgg.h5')

In [ ]:
from keras.models import load_model

classifier = load_model('D:\OpenVINO\cellsvgg.h5')

In [ ]:
rand = np.random.randint(0, samples)
input_im = X[rand]
imshow((input_im).astype('uint8'))
input_im = input_im.reshape(1, im_size[0], im_size[1], im_size[2])
res = str(model.predict_classes(input_im, 1, verbose = 0)[0])

y_test_labels = y[to_train:]

plt.title(names[rand] + '. Predicted: ' + (res) + ' Real: ' + str(int(y[rand])))

In [ ]:
for i in np.random.permutation(samples)[:10]:
    input_im = X[i]
    input_im = input_im.reshape(1, im_size[0], im_size[1], im_size[2])
    res = str(model.predict_classes(input_im, 1, verbose = 0)[0])
    print(names[i], "predicted:", res, "real:", int(y[i]))

In [ ]:
x_test[0].shape